In [1]:
from tqdm import tqdm

# Utilisation dans une boucle
for i in tqdm(range(100)):
    pass


100%|██████████| 100/100 [00:00<00:00, 480998.17it/s]


### Embeddings pour le Catalogue de Produits

In [2]:
from transformers import AutoTokenizer, AutoModel
import pandas as pd

# Charger le modèle Hugging Face
model_name = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

# Charger le catalogue de produits
product_catalog = pd.read_csv("/mnt/c/Users/amavi/projects-studio/ecommerce/AI-System/data_science/csv_stock/product_catalog.csv")

# Fonction pour générer les embeddings
def get_embedding(text):
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True)
    outputs = model(**inputs)
    return outputs.last_hidden_state.mean(dim=1).detach().numpy()

# Générer les embeddings pour chaque produit
product_catalog['embedding'] = product_catalog['description'].apply(get_embedding)
print(product_catalog)
# Sauvegarder les embeddings (optionnel)
product_catalog.to_pickle("/mnt/c/Users/amavi/projects-studio/ecommerce/AI-System/data_science/csv_stock/product_catalog_embeddings.pkl")


   id          name                                        description  \
0   1  Gaming Mouse        Wireless gaming mouse with ergonomic design   
1   2        Laptop   High-performance gaming laptop with RGB lighting   
2   3    Headphones  Noise-canceling over-ear headphones with high ...   
3   4      Keyboard    Mechanical keyboard with customizable backlight   
4   5       Monitor          Ultra HD 4K monitor with adjustable stand   

                                           embedding  
0  [[-0.24534485, -0.13033515, 0.16920622, 0.0787...  
1  [[-0.35070655, -0.22071186, 0.3561776, 0.00902...  
2  [[0.044147037, 0.084562525, 0.22871184, 0.1906...  
3  [[-0.21754181, -0.13643745, 0.29902977, 0.1873...  
4  [[0.02546438, -0.28171644, 0.581398, 0.1719977...  


### Chaîne LangChain pour les Recommandations

In [3]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

# Charger les données utilisateur-produit et le catalogue de produits
interactions = pd.read_csv("/mnt/c/Users/amavi/projects-studio/ecommerce/AI-System/data_science/csv_stock/user_product_interactions.csv")
product_catalog = pd.read_pickle("/mnt/c/Users/amavi/projects-studio/ecommerce/AI-System/data_science/csv_stock/product_catalog_embeddings.pkl")

# Vérifier et convertir les embeddings produits en vecteurs 1D numpy
def preprocess_embeddings(catalog):
    """
    Prépare les embeddings pour s'assurer qu'ils sont tous au bon format.
    """
    catalog["embedding"] = catalog["embedding"].apply(
        lambda emb: np.array(emb).squeeze()  # Convertir en numpy et retirer les dimensions inutiles
    )
    return catalog

# Prétraiter les embeddings du catalogue de produits
product_catalog = preprocess_embeddings(product_catalog)

# Fonction pour extraire les préférences utilisateur
def extract_user_preferences(user_id: int):
    """
    Extrait les préférences d'un utilisateur à partir de ses interactions.
    """
    user_data = interactions[interactions['user_id'] == user_id]
    if user_data.empty:
        return None, f"Aucune interaction trouvée pour l'utilisateur {user_id}."

    # Identifier les produits préférés
    preferred_products = product_catalog.loc[
        product_catalog['id'].isin(user_data['product_id'])
    ]
    if preferred_products.empty:
        return None, f"Aucun produit correspondant trouvé pour l'utilisateur {user_id}."

    # Calculer l'embedding moyen pour l'utilisateur
    embeddings = np.array(preferred_products['embedding'].tolist())
    user_embedding = np.mean(embeddings, axis=0)  # Moyenne des embeddings
    return user_embedding, None

# Fonction pour recommander des produits
def recommend_products(user_embedding: np.ndarray):
    """
    Recommande des produits similaires en fonction de l'embedding utilisateur.
    """
    if user_embedding is None:
        return [], "Embedding utilisateur non défini."

    # Assurer que `user_embedding` est 2D pour cosine_similarity
    user_embedding = np.expand_dims(user_embedding, axis=0)

    # Calculer la similarité cosinus pour chaque produit
    def calculate_similarity(emb):
        emb = np.expand_dims(emb, axis=0)  # Assurer que chaque embedding produit est 2D
        return cosine_similarity(user_embedding, emb)[0][0]

    product_catalog["similarity"] = product_catalog["embedding"].apply(calculate_similarity)

    # Trier les produits par similarité
    recommendations = product_catalog.sort_values(by="similarity", ascending=False).head(5)

    if recommendations.empty:
        return [], "Aucune recommandation trouvée."

    return recommendations[['id', 'name', 'similarity']].to_dict(orient='records'), None


### Pipeline de recommandations


In [4]:
# Pipeline de recommandations
def run_recommendation_pipeline(user_id: int):
    """
    Exécute le pipeline de recommandations pour un utilisateur spécifique.
    """
    # Étape 1 : Extraire les préférences utilisateur
    user_embedding, error = extract_user_preferences(user_id)
    if error:
        return {"error": error}

    # Étape 2 : Recommander des produits
    recommendations, error = recommend_products(user_embedding)
    if error:
        return {"error": error}

    return {"recommendations": recommendations}

### Tester le pipeline


In [7]:
# Tester le pipeline
if __name__ == "__main__":
    user_id_test = 2  # Remplace avec un ID utilisateur valide
    result = run_recommendation_pipeline(user_id_test)

    if "error" in result:
        print(f"Erreur : {result['error']}")
    else:
        print(f"Recommandations pour l'utilisateur {user_id_test} :")
        for rec in result["recommendations"]:
            print(f"- {rec['name']} (similarité : {rec['similarity']:.2f})")

Recommandations pour l'utilisateur 2 :
- Keyboard (similarité : 0.95)
- Headphones (similarité : 0.95)
- Laptop (similarité : 0.88)
- Gaming Mouse (similarité : 0.87)
- Monitor (similarité : 0.86)
